# Why  are some US counties more affected than others? Exploring the spread of Covid-19 in the US counties



<img src='../Data/images/cv.jpg'>

## Import the Libraries

In [72]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta # date time

#Importing Data plotting libraries
import matplotlib.pyplot as plt     
import seaborn as sns      
import plotly.express as px
import plotly.graph_objects as go

## Load the Datasets

> Source: <a href= "https://github.com/nytimes/covid-19-data"> NY-Times Covid-19 Data </a>

In [73]:
'''
Potential Questions?
1. Why are some counties more affected than the other?
2. Are the counties in low income bracket more affected?
'''
state_level = pd.read_csv("../Data/NY_times/us-states.csv")

## Let's Analyze the state-level data

In [74]:
# No null values in the data
state_level.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5569 entries, 0 to 5568
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    5569 non-null   object
 1   state   5569 non-null   object
 2   fips    5569 non-null   int64 
 3   cases   5569 non-null   int64 
 4   deaths  5569 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 217.7+ KB


In [75]:
state_level.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [76]:
print("The data we have is from {} to {}".format(state_level['date'].min(), state_level['date'].max()))
print("The date on which the first covid-19 case was recorded: {}".format(state_level[state_level['cases']>0]['date'].iloc[0]))
print("The date on which the first death from covid-19 was recorded: {}".format(state_level[state_level['deaths']>0]['date'].iloc[0]))

The data we have is from 2020-01-21 to 2020-06-11
The date on which the first covid-19 case was recorded: 2020-01-21
The date on which the first death from covid-19 was recorded: 2020-02-29


### Let's convert state names to state codes (abbreviations)

In [77]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

state_level['state_code'] = state_level['state'].map(us_state_abbrev)
state_level.head()

,date,state,fips,cases,deaths,state_code
0,2020-01-21,Washington,53,1,0,WA
1,2020-01-22,Washington,53,1,0,WA
2,2020-01-23,Washington,53,1,0,WA
3,2020-01-24,Illinois,17,1,0,IL
4,2020-01-24,Washington,53,1,0,WA


### Plotting a Running Map for observing the spread of COVID-19 Confirmed Cases

In [78]:
statewise_group = state_level.groupby(['date', 'state_code', 'fips'])[['cases', 'deaths']].max().reset_index()
statewise_group['ob_date'] = pd.to_datetime(statewise_group['date'])
statewise_group['ob_date'] = statewise_group['ob_date'].dt.strftime('%m/%d/%Y')
statewise_group = statewise_group.sort_values('ob_date')

statewise_group.loc[:, 'log_ConfirmedCases'] = np.log(statewise_group.cases + 1)

fig = px.choropleth(locations=statewise_group['state_code'],
                    color=statewise_group["log_ConfirmedCases"], 
                    locationmode="USA-states",
                    scope="usa",
                    animation_frame=statewise_group['ob_date'],
                    color_continuous_scale=px.colors.sequential.Viridis,
                   )

layout = go.Layout(
    title=go.layout.Title(
        text="The spread of Covid-19 cases in the US states",
        x=0.5
    ),
    font=dict(size=14),
)

fig.update_layout(layout)
fig.show()

### Plotting a Running Map for observing the fatalities of COVID-19 

In [19]:
statewise_deaths = statewise_group[statewise_group['deaths']>0]

fig = px.choropleth(locations=statewise_deaths['state_code'],
                    color=statewise_deaths["deaths"], 
                    locationmode="USA-states",
                    scope="usa",
                    animation_frame=statewise_deaths['ob_date'],
                    color_continuous_scale=px.colors.sequential.Viridis
                   )

layout = go.Layout(
    title=go.layout.Title(
        text="The deaths in the US states due to Covid-19",
        x=0.5
    ),
    font=dict(size=14),
)

fig.update_layout(layout)
fig.show()


## Let's Analyze county-level data

> Source: 
1. <a href="https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/"> USA Facts </a>
2. <a href="https://www.kaggle.com/roche-data-science-coalition/uncover?"> United Network for COVID Data Exploration and Research </a>
3. <a href = "https://www.ers.usda.gov/data-products.aspx"> USDA ERS </a>

In [47]:
today_date = datetime.today()- timedelta(days=2)
today_date = today_date.strftime('%m/%d/%y')[1:]

'''
Get the relevant columns only.
Set the same column names for all the county-level data. Easy of use.
'''
county_cases = pd.read_csv("../Data/USA_facts/covid_confirmed_usafacts.csv")[['countyFIPS', 'County Name', 'State', today_date]]
county_cases.columns = ['cfips', 'county', 'state', 'cases']
county_cases = county_cases.set_index('cfips')

county_deaths = pd.read_csv("../Data/USA_facts/covid_deaths_usafacts.csv")[['countyFIPS', today_date]]
county_deaths.columns = ['cfips', 'deaths']
county_deaths = county_deaths.set_index('cfips')

county_population = pd.read_csv("../Data/USA_facts/covid_county_population_usafacts.csv")[['countyFIPS', 'population']]
county_population.columns = ['cfips', 'population']
county_population = county_population.set_index('cfips')

county_level = county_cases.join(county_deaths)
county_level = county_level.join(county_population)

county_level = county_level[county_level.index>999]
county_level = county_level[county_level['population']>0]

del county_cases, county_population, county_deaths

#Let's add a few more details such as mortality, deaths per million, cases per million 
county_level['mortality'] = county_level['deaths']/county_level['cases']
county_level['mortality'] = county_level['mortality'].fillna(0) # in case of null values

county_level['deaths_per_mil'] = county_level['deaths'] * 1000000/county_level['population']
county_level['cases_per_mil'] = county_level['cases'] * 1000000/county_level['population']

county_level = county_level.reset_index()

county_level.head()

,cfips,county,state,cases,deaths,population,mortality,deaths_per_mil,cases_per_mil
0,1001,Autauga County,AL,312,6,55869.0,0.019231,107.394083,5584.492294
1,1003,Baldwin County,AL,343,9,223234.0,0.026239,40.316439,1536.504296
2,1005,Barbour County,AL,214,1,24686.0,0.004673,40.508790,8668.881147
3,1007,Bibb County,AL,93,1,22394.0,0.010753,44.654818,4152.898098
4,1009,Blount County,AL,87,1,57826.0,0.011494,17.293259,1504.513541


In [81]:
# Adding more county level data to understand the Covid-19 spread in the US counties
county_health = pd.read_csv("../Data/uncover/us-county-health-rankings-2020.csv")[['fips', 'percent_smokers', 
'percent_adults_with_obesity', 'percent_physically_inactive', 'percent_excessive_drinking', 'percent_uninsured',
'num_primary_care_physicians', 'num_mental_health_providers', 'high_school_graduation_rate', 'percent_some_college',
'percent_unemployed', 'percent_children_in_poverty', 'life_expectancy']]
county_health = county_health.rename(columns={'fips': 'cfips'})

county_poverty = pd.read_csv("../Data/USDA/PovertyEstimates.csv")[["FIPStxt", "PCTPOVALL_2018", "MEDHHINC_2018"]]
county_poverty = county_poverty.rename(columns={'FIPStxt': 'cfips', 'PCTPOVALL_2018': 'percent_poverty', 
                                               'MEDHHINC_2018': 'median_hincome'})

county_level = county_level.merge(county_health)
county_level = county_level.merge(county_poverty)
county_level.head()

,cfips,county,state,cases,deaths,population,mortality,deaths_per_mil,cases_per_mil,percent_smokers,...,percent_uninsured,num_primary_care_physicians,num_mental_health_providers,high_school_graduation_rate,percent_some_college,percent_unemployed,percent_children_in_poverty,life_expectancy,percent_poverty,median_hincome
0,1001,Autauga County,AL,312,6,55869.0,0.019231,107.394083,5584.492294,18.081557,...,8.721686,25.0,13.0,90.000000,62.009974,3.629079,19.3,76.879477,13.8,"59,338"
1,1003,Baldwin County,AL,343,9,223234.0,0.026239,40.316439,1536.504296,17.489033,...,11.333404,155.0,210.0,86.361577,67.371620,3.615382,13.9,78.450258,9.8,"57,588"
2,1005,Barbour County,AL,214,1,24686.0,0.004673,40.508790,8668.881147,21.999985,...,12.242792,8.0,2.0,81.410256,34.857649,5.171384,43.9,75.341935,30.9,"34,382"
3,1007,Bibb County,AL,93,1,22394.0,0.010753,44.654818,4152.898098,19.114200,...,10.206253,11.0,5.0,83.763838,44.137353,3.971828,27.8,73.571820,21.8,"46,064"
4,1009,Blount County,AL,87,1,57826.0,0.011494,17.293259,1504.513541,19.208672,...,13.360759,13.0,9.0,93.468795,53.361073,3.511157,18.0,74.145826,13.2,"50,412"


### Counties with most cases

In [123]:
cases_most = county_level.groupby(['county', 'state'])['cases'].max().reset_index()
cases_most = cases_most.sort_values('cases', ascending=False).head(10)

fig = px.bar(cases_most, x='cases', y='county', color='state')
fig.update_layout(
    title="Counties with most number of Cases",
    xaxis_title="Total number of cases",
    yaxis_title="Counties",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f")
)
fig.show()

### Counties with most cases per million

In [122]:
cases_per_mil = county_level.groupby(['county', 'state'])['cases_per_mil'].max().reset_index()
cases_per_mil = cases_per_mil.sort_values('cases_per_mil', ascending=False).head(10)

fig = px.bar(cases_per_mil, x='cases_per_mil', y='county', color='state')
fig.update_layout(
    title="Counties with most number of Cases per million",
    xaxis_title="cases per million",
    yaxis_title="Counties",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f")
)
fig.show()

### Counties with most deaths

In [125]:
deaths_most = county_level.groupby(['county', 'state'])['deaths'].max().reset_index()
deaths_most = deaths_most.sort_values('deaths', ascending=False).head(10)

fig = px.bar(deaths_most, x='deaths', y='county', color='state')
fig.update_layout(
    title="Counties with most number of Deaths",
    xaxis_title="Total number of deaths",
    yaxis_title="Counties",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f")
)
fig.show()

### Does Poverty percentage affect the spread of Covid-19

In [110]:
poverty = county_level.groupby(['county', 'state'])[['cases', 'deaths', 'percent_unemployed', 'percent_children_in_poverty',
                                                       'percent_poverty', 'median_hincome']].max().reset_index()
least_poverty = poverty.sort_values('percent_poverty').head(10)
most_poverty = poverty.sort_values('percent_poverty', ascending = False).head(10)

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=least_poverty['county'], y=least_poverty['cases'],
                    mode='lines',
                    name='Least Poverty percent counties'))
fig.add_trace(go.Scatter(x=most_poverty['county'], y=most_poverty['cases'],
                    mode='lines+markers',
                    name='Most Poverty percent counties'))
# Edit the layout
fig.update_layout(title='Variation of Covid-19 cases among the top 10 counties with lowest and highest poverty percent',
                   xaxis_title='Counties',
                   yaxis_title='Total number of Cases')
fig.show()



In [116]:
#Assigning a threshold of 25% poverty index to analyze the spread of COVID-19 among these counties
below_25 = poverty[poverty['percent_poverty']<=25]
below_25 =below_25.sort_values('percent_poverty')

fig = px.line(below_25, x='percent_poverty', y='cases')
fig.update_layout(
    title="Cases in counties with poverty percentage below 25",
    xaxis_title="Poverty Percentage",
    yaxis_title="Total number of Cases",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f")
)
fig.show()


In [117]:
above_25 = poverty[poverty['percent_poverty']>25]
above_25 = above_25.sort_values('percent_poverty')

fig = px.line(above_25, x='percent_poverty', y='cases')
fig.update_layout(
    title="Cases in counties with poverty percentage below 25",
    xaxis_title="Poverty Percentage",
    yaxis_title="Total number of Cases",
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f")
)
fig.show()